In [ ]:
!pip install -q -U trax

     |████████████████████████████████| 368kB 3.5MB/s 
     |████████████████████████████████| 2.6MB 11.0MB/s 
     |████████████████████████████████| 163kB 33.9MB/s 
     |████████████████████████████████| 1.5MB 31.3MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 1.1MB 41.9MB/s 
     |████████████████████████████████| 307kB 40.4MB/s 
     |████████████████████████████████| 778kB 39.9MB/s 
     |████████████████████████████████| 3.5MB 41.9MB/s 
     |████████████████████████████████| 358kB 37.9MB/s 
     |████████████████████████████████| 368kB 40.2MB/s 
     |████████████████████████████████| 194kB 36.3MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 5.3MB 33.6MB/s 
     |████████████████████████████████| 655kB 28.3MB/s 
     |████████████████████████████████| 983kB 35.0MB/s 
     |████████████████████████████████| 890kB 25.6MB/s 
     |████████████████████████████████| 3.0MB 26.9MB/

In [ ]:
import pandas as pd
import numpy as np
import trax
import trax.fastmath.numpy as fastnp
import trax.layers as tl
import random

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 


In [ ]:
lines = []
with open('t8.shakespeare.txt') as file:
  for line in file:
    p_line = line.strip()
    if p_line.isdigit() == False and p_line:
      lines.append(p_line)
print(len(lines))
print(lines[0])

114443
From fairest creatures we desire increase,


In [ ]:
lines = [x.lower() for x in lines]
lines[0]

'from fairest creatures we desire increase,'

In [ ]:
train_data = lines[:-3000]
eval_data = lines[-3000:]
print(train_data[0])
print(eval_data[0])
print(len(train_data))
print(len(eval_data))

from fairest creatures we desire increase,
that vulgars give bold'st titles; ay, and privy
111443
3000


In [ ]:
def text_to_tensor(text,End_of_line = 1):
  tensor = []
  for char in text:
    tensor.append(ord(char))
  tensor.append(End_of_line)
  return tensor

In [ ]:
#test
text = 'from fairest creatures we desire increase,'
print(text_to_tensor(text))

[102, 114, 111, 109, 32, 102, 97, 105, 114, 101, 115, 116, 32, 99, 114, 101, 97, 116, 117, 114, 101, 115, 32, 119, 101, 32, 100, 101, 115, 105, 114, 101, 32, 105, 110, 99, 114, 101, 97, 115, 101, 44, 1]


In [ ]:
def datagenerator(datalines,batch_size,shuffle=True,pad = 0):
  index = 0
  x = []
  len_index_lines = len(datalines)
  index_lines = list(range(len_index_lines))

  if shuffle:
    random.shuffle(index_lines)

  while True:
    if index >= len_index_lines:
      index = 0
      if shuffle:
        random.shuffle(index_lines)
    
    line = datalines[index_lines[index]]
    x.append(text_to_tensor(line))
    index += 1

    if (batch_size == len(x)):
      temp_x = []
      mask = []
      max_len = 0
      for i in range(batch_size):
        if max_len < len(x[i]):
          max_len = len(x[i])
      for i in range(batch_size):
        temp = x[i] + [pad]*(max_len-len(x[i]))
        temp_x.append(temp)
        example_mask = [1 if i!=0 else 0 for i in temp]
        mask.append(example_mask)
      final_x = fastnp.array(temp_x)
      final_mask = fastnp.array(mask)
      yield final_x,final_x,final_mask
      x = []

In [ ]:
import itertools
batch_size = 32
train_generator = itertools.cycle(datagenerator(train_data,batch_size))
eval_generator = itertools.cycle(datagenerator(eval_data,batch_size))

In [ ]:
def language_model(vocab_size = 256, embed_size = 512 , mode = 'train' ):
  model = tl.Serial(tl.ShiftRight(mode = mode),tl.Embedding(vocab_size,embed_size),tl.LSTM(n_units=embed_size),tl.Dense(vocab_size),tl.LogSoftmax())
  return model

In [ ]:
from trax.supervised import training

In [ ]:
def train_model(language_model,train_generator,eval_generator,n_steps = 1,output_dir = '/model'):
  train_task = training.TrainTask(train_generator,tl.CrossEntropyLoss(),trax.optimizers.Adam(0.01))
  eval_task = training.EvalTask(eval_generator,[tl.CrossEntropyLoss(),tl.Accuracy()],n_eval_batches=3)

  train_loop = training.Loop(language_model,train_task,eval_tasks=[eval_task],output_dir = output_dir)
  train_loop.run(n_steps)
  return train_loop

In [ ]:
loop = train_model(language_model(),train_generator,eval_generator,n_steps=7000)


Step      1: Ran 1 train steps in 7.21 secs
Step      1: train CrossEntropyLoss |  5.83509970
Step      1: eval  CrossEntropyLoss |  4.72332557
Step      1: eval          Accuracy |  0.16387086

Step    100: Ran 99 train steps in 217.45 secs
Step    100: train CrossEntropyLoss |  2.45340681
Step    100: eval  CrossEntropyLoss |  2.27326918
Step    100: eval          Accuracy |  0.32806053

Step    200: Ran 100 train steps in 202.64 secs
Step    200: train CrossEntropyLoss |  2.15493989
Step    200: eval  CrossEntropyLoss |  2.10615532
Step    200: eval          Accuracy |  0.36725548

Step    300: Ran 100 train steps in 194.75 secs
Step    300: train CrossEntropyLoss |  2.06334257
Step    300: eval  CrossEntropyLoss |  2.04981260
Step    300: eval          Accuracy |  0.39338951

Step    400: Ran 100 train steps in 193.26 secs
Step    400: train CrossEntropyLoss |  2.00005269
Step    400: eval  CrossEntropyLoss |  1.99108402
Step    400: eval          Accuracy |  0.40075245

Step    5

In [ ]:
model = language_model()
#model.init_from_file('/model/model.pkl.gz')
model.init_from_file('model.pkl.gz')

In [ ]:
#log perplexity
def perplexity(predictions,targets):
  total = fastnp.sum(predictions * tl.one_hot(targets,predictions.shape[-1]),axis=-1)
  non_pad = 1.0 - fastnp.equal(targets,0)
  non_pad_ppx = total*non_pad
  return -fastnp.sum(non_pad_ppx)/np.sum(non_pad)

In [ ]:
batch = next(datagenerator(train_data,batch_size))
pred = model(batch[0])
ppx = perplexity(pred,batch[1])
print('perplexity',ppx)

perplexity 1.5439323


In [ ]:
def gumbel_sample(log_probs, temperature=1.0):
    """Gumbel sampling from a categorical distribution."""
    u = np.random.uniform(low=1e-6, high=1.0 - 1e-6, size=log_probs.shape)
    g = -fastnp.log(-np.log(u))
    return fastnp.argmax(log_probs + g * temperature, axis=-1)

In [ ]:
#test
text = "From f"
tensor = text_to_tensor(text)
tensor = fastnp.array(tensor)
pred = model(tensor[None,:])
nextchar = gumbel_sample(pred[0,len(text)])
print(nextchar)
print(chr(int(nextchar)))


111
o


In [ ]:
def predict(n_chars,prefix):
  input = [ord(c) for c in prefix]
  result = [c for c in prefix]
  max_len = len(prefix)+n_chars
  for i in range(n_chars):
    curr_input = fastnp.array(input + [0]*(max_len - len(input)))
    output = model(curr_input[None,:])
    next_char = gumbel_sample(output[0,len(input)])
    input += [int(next_char)]

    if len(input) == -1:
      break
    result.append(chr(int(next_char)))
  return "".join(result)

In [ ]:
text = predict(32,"")
print(text)

to such wife, much rose, a king 
